In [ ]:
!pip install mlflow dagshub -qqq

In [ ]:
import mlflow
import dagshub

dagshub.init(
    repo_name="mlflow_major",
    repo_owner="iambikash378",
    mlflow=True
)

mlflow.set_tracking_uri(f"https://dagshub.com/iambikash378/mlflow_major.mlflow")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=d0df08b0-1a42-4b92-9b83-83b75c1a9232&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=bbb40aa79807a85e67ed84ed59e5ce6e393e1b6b42483d20a4604442802d4e41




Output()

Accessing as iambikash378

Initialized MLflow to track repo "iambikash378/mlflow_major"

Repository iambikash378/mlflow_major initialized!

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
%cd /content/drive/MyDrive/major_proj

/content/drive/.shortcut-targets-by-id/1Ksve6Nua5d8mXhXEQl0u5pVhxS2-eWv5/major_proj


In [ ]:
!pip install -U segmentation-models-pytorch

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import segmentation_models_pytorch as smp

model = smp.DeepLabV3Plus(
    encoder_name="resnet34",
    encoder_depth = 5,
    encoder_weights="imagenet",
    in_channels= 4,
    classes= 1,
)
model= model.to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 83.6MB/s]


In [ ]:
from segmentation_models_pytorch.encoders import get_preprocessing_fn

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [ ]:
from torch.utils.data import Dataset
import torch
import albumentations as A
import numpy as np

# # Define the augmentation pipeline
# transform = A.Compose([
#     A.HorizontalFlip(p=0.5),   # Horizontal flip
#     A.VerticalFlip(p=0.5),     # Vertical flip (if you want both)
#     A.RandomRotate90(p=0.5),   # Random 90-degree rotation
#     A.RandomCrop(width=128, height=128, p=1.0),  # Crop image
#     A.Rotate(limit=45, p=1.0),
# ])

class hrgldd_dataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        img = self.x[index]
        label = self.y[index]

        img = torch.from_numpy(img).float().permute(2,0,1)
        label = torch.from_numpy(label).float().permute(2,0,1)

        return img, label


/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
import numpy as np
from torch.utils.data import DataLoader

path_to_testX = '/content/drive/MyDrive/major_proj/HR-GLDD/testX.npy'
path_to_testY = '/content/drive/MyDrive/major_proj/HR-GLDD/testY.npy'

path_to_trainX = '/content/drive/MyDrive/major_proj/HR-GLDD/jointX.npy'
path_to_trainY = '/content/drive/MyDrive/major_proj/HR-GLDD/jointY.npy'

path_to_valX = '/content/drive/MyDrive/major_proj/HR-GLDD/valX.npy'
path_to_valY = '/content/drive/MyDrive/major_proj/HR-GLDD/valY.npy'

data_testY = np.load(path_to_testY)
data_testX = np.load(path_to_testX)

data_trainY = np.load(path_to_trainY)
data_trainX = np.load(path_to_trainX)

data_valY = np.load(path_to_valY)
data_valX = np.load(path_to_valX)

train_dataset = hrgldd_dataset(data_trainX, data_trainY)
val_dataset = hrgldd_dataset(data_valX, data_valY)
test_dataset = hrgldd_dataset(data_testX, data_testY)

In [ ]:
LEARNING_RATE = 5e-4
BATCH_SIZE = 16
EPOCHS = 500
PATIENCE = 20
MODEL_SAVE_PATH = "/content/drive/MyDrive/major_proj/trained_models/deeplabv3plus.pth"
THRESHOLD = 0.5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


train_dataloader = DataLoader(dataset=train_dataset,
                               batch_size=BATCH_SIZE,
                               shuffle=True)
val_dataloader = DataLoader(dataset=val_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True)
train_dataloader = DataLoader(dataset=train_dataset,
                               batch_size=BATCH_SIZE,
                               shuffle=True)

In [ ]:
preprocess_input = get_preprocessing_fn('resnet18', pretrained='imagenet')

In [ ]:
from segmentation_models_pytorch import losses
from segmentation_models_pytorch.losses.constants import BINARY_MODE
loss_fn = losses.FocalLoss(mode = BINARY_MODE)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

In [ ]:
mlflow.end_run()

In [ ]:
best_val_loss = float('inf')
counter = 0

with mlflow.start_run():
  mlflow.log_params(
      {
          "initial_learning_rate": LEARNING_RATE,
          "batch_size": BATCH_SIZE,
          "model_arch": 'DeepLabV3Plus',
          "encoder_name": 'resnet34',
          "encoder_depth": 5,
          "encoder_weights": 'imagenet',
          "loss_function": loss_fn,
          "epochs": EPOCHS,
          "patience": PATIENCE,
          "optimizer": type(optimizer).__name__,
          "threshold" : THRESHOLD,
          "remarks" : "augmented"
      }
  )
  for epoch in range(EPOCHS):
    train_running_loss = 0
    model.train()
    for images, gt_masks in train_dataloader:

        images = images.to(device)
        gt_masks = gt_masks.to(device)

        optimizer.zero_grad()

        predicted_mask = model(images)
        loss = loss_fn(predicted_mask, gt_masks)

        loss.backward()
        optimizer.step()

        train_running_loss += loss.item()
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3)

    train_loss = train_running_loss / len(train_dataloader)
    print(f"\nEpoch: {epoch+1}/{EPOCHS}, Train Loss: {train_loss}")
    mlflow.log_metric("train_loss", train_loss, step=epoch)

    #############################################################

    model.eval()
    val_running_loss = 0
    #val_dice = 0
    val_accuracy = 0
    val_precision = 0
    val_recall = 0
    val_f1 = 0
    with torch.no_grad():
        for images, gt_masks in val_dataloader:

            images = images.to(device)
            gt_masks = gt_masks.to(device)

            y_pred = model(images)
            val_loss = loss_fn(y_pred, gt_masks)
            val_running_loss += val_loss.item()


            y_pred_binary = (torch.sigmoid(y_pred) > THRESHOLD).float()
            mask_binary = (gt_masks > 0.5).float()
            y_pred_flat = y_pred_binary.view(-1).cpu().numpy()
            mask_flat = mask_binary.view(-1).cpu().numpy()

            val_accuracy += accuracy_score(mask_flat, y_pred_flat)
            val_precision += precision_score(mask_flat, y_pred_flat, zero_division=0)
            val_recall += recall_score(mask_flat, y_pred_flat, zero_division=0)
            val_f1 += f1_score(mask_flat, y_pred_flat, zero_division=0)

        val_loss = val_running_loss / len(val_dataloader)
        #val_loss_list.append(val_loss)
        #val_dice = val_dice / len(val_dataloader)
        val_accuracy = val_accuracy / len(val_dataloader)
        val_precision = val_precision / len(val_dataloader)
        val_recall = val_recall / len(val_dataloader)
        val_f1 = val_f1 / len(val_dataloader)

        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("val_accuracy", val_accuracy, step=epoch)
        mlflow.log_metric("val_precision", val_precision, step=epoch)
        mlflow.log_metric("val_recall", val_recall, step=epoch)
        mlflow.log_metric("val_f1", val_f1, step=epoch)

    print("--" * 30)
    print(f"Train Loss EPOCH {epoch + 1}: {train_loss:.4f}")
    print(f"Val Loss EPOCH {epoch + 1} : {val_loss:.4f}")
    #print(f"Val Dice Score EPOCH {epoch + 1} : {val_dice:.4f}")
    print(f"Val Precision EPOCH {epoch + 1}: {val_precision:.4f}")
    print(f"Val Recall EPOCH {epoch + 1}: {val_recall:.4f}")
    print(f"Val F1 Score EPOCH {epoch + 1}: {val_f1:.4f}")
    print(f"Current Learning Rate : {optimizer.param_groups[0]['lr']:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
        mlflow.log_artifact(MODEL_SAVE_PATH)
        print(f"Saved best model with val loss : {val_loss:.4f}")
    else:
        counter += 1
        print(f"No improvement in validation loss for {counter} epochs.")

    if counter >= PATIENCE:
        print(f"Early stopping triggered after {PATIENCE} epochs without improvement.")
        break

    scheduler.step(val_loss)

